In [1]:
import torch
import json
import numpy as np
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [2]:
# Set random seed for reproducibility
seed_val = 42
np.random.seed(seed_val)
torch.manual_seed(seed_val)

In [3]:
# Load Spoken SQuAD dataset
with open('spoken_train.json', 'r') as f:
    squad_data_trian = json.load(f)

with open('spoken_test.json', 'r') as f:
    squad_data_test = json.load(f)

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',truncation='do_not_truncate')

In [5]:
class SquadDataset(Dataset):
    def __init__(self, squad_data, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.questions = []
        self.contexts = []
        self.answers = []

        for article in squad_data['data']:
            for paragraph in article['paragraphs']:
                context = paragraph['context']
                for qa in paragraph['qas']:
                    question = qa['question']
                    answer_text = qa['answers'][0]['text']
                    answer_start = qa['answers'][0]['answer_start']
                    answer_end = answer_start + len(answer_text)

                    # Create tokenized input and target answer spans
                    tokenized_inputs = tokenizer.encode_plus(question, context, max_length=max_len, padding='max_length', truncation=True, return_tensors='pt')
                    token_start = tokenized_inputs['input_ids'][0].tolist().index(tokenizer.sep_token_id) + 1
                    token_end = token_start + len(self.tokenizer.tokenize(context[answer_start:answer_end])) - 1

                    self.questions.append(question)
                    self.contexts.append(context)
                    self.answers.append({'answer_start': answer_start, 'answer_end': answer_end, 'token_start': token_start, 'token_end': token_end})

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        inputs = self.tokenizer.encode_plus(self.questions[idx], self.contexts[idx], max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')
        return {'input_ids': inputs['input_ids'][0], 'attention_mask': inputs['attention_mask'][0], 'start_positions': self.answers[idx]['token_start'], 'end_positions': self.answers[idx]['token_end']}

In [6]:
batch_size = 16
max_len = 387
num_epochs = 10
learning_rate = 2e-5

model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
optimizer = AdamW(model.parameters(), lr=learning_rate)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [7]:
train_on_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
train_dataset = SquadDataset(squad_data_trian, tokenizer, max_len)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [46]:
for epoch in range(num_epochs):
    model.train()
    print(f'Epoch {epoch + 1}/{num_epochs}')
    print('-' * 10)
    train_loss = 0.0
    train_steps = 0
    for batch in tqdm(train_dataloader):
        # Move batch to GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        # Zero out gradients
        optimizer.zero_grad()

        # Forward pass
        model.to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)#.to(device)
        loss = outputs.loss#.to(device)

        # Backward pass
        loss.backward()

    # Update weights
        optimizer.step()#.to(device)

    # Update training loss
        train_loss += loss.item()
        train_steps += 1

    # Compute average training loss
    avg_train_loss = train_loss / train_steps
    print(f'Epoch {epoch+1} - Avg training loss: {avg_train_loss:.4f}') 


Epoch 1/10
----------


  2%|▏         | 36/2320 [00:15<16:10,  2.35it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  5%|▌         | 117/2320 [00:49<15:35,  2.35it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been remove

Epoch 1 - Avg training loss: 0.9734
Epoch 2/10
----------


 10%|█         | 240/2320 [01:43<14:55,  2.32it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 11%|█         | 257/2320 [01:50<14:50,  2.32it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been remov

Epoch 2 - Avg training loss: 0.8377
Epoch 3/10
----------


...

Epoch 8 - Avg training loss: 0.2217
Epoch 9/10
----------


 11%|█         | 259/2320 [01:49<14:33,  2.36it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 25%|██▍       | 569/2320 [04:01<12:21,  2.36it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been remov

Epoch 9 - Avg training loss: 0.1762
Epoch 10/10
----------


 22%|██▏       | 500/2320 [03:34<12:59,  2.33it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 36%|███▌      | 838/2320 [05:59<10:40,  2.32it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been remov

Epoch 10 - Avg training loss: 0.1446


In [47]:
test_dataset = SquadDataset(squad_data_test, tokenizer, max_len)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [48]:
model.eval()
total_loss = 0
predictions = []
true_start = []
true_end = []
pred_true_start = []
pred_true_end = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits
        
        '''print('start_scores:')
        print(start_scores)
        print('end_scores:')
        print(end_scores)'''

        answer_start = torch.argmax(start_scores)
        answer_end = torch.argmax(end_scores)
        
        '''print('answer_start:')
        print(answer_start)
        print('answer_end:')
        print(answer_end)'''
        print('Answer: position ' + str(answer_start.item()) + ' to ' + str(answer_end.item()))


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Answer: position 1171 to 3493
Answer: position 395 to 397
Answer: position 5821 to 1177
Answer: position 5822 to 2718
Answer: position 2724 to 3113
Answer: position 12 to 399
Answer: position 785 to 414
Answer: position 4656 to 4270
Answer: position 400 to 1174
Answer: position 3106 to 4660
Answer: position 5431 to 5432
Answer: position 794 to 4663
Answer: position 4654 to 4278
Answer: position 20 to 22
Answer: position 400 to 3112
Answer: position 2335 to 5041
Answer: position 4658 to 2726
Answer: position 3890 to 3892
Answer: position 2336 to 23
Answer: position 3115 to 4660
Answer: position 5818 to 5819
Answer: position 1561 to 1562
Answer: position 4272 to 1954
Answer: position 3496 to 4272
Answer: position 1174 to 2331
Answer: position 3886 to 786
Answer: position 1566 to 1172
Answer: position 1175 to 1177
Answer: position 3881 to 3111
Answer: position 2331 to 3888
Answer: position 3886 to 3887
Answer: position 5817 to 23
Answer: position 1949 to 3879
Answer: position 1558 to 5431

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Answer: position 5042 to 13
Answer: position 5822 to 1178
Answer: position 14 to 15
Answer: position 3885 to 5043
Answer: position 1561 to 5822
Answer: position 5041 to 2724
Answer: position 15 to 16
Answer: position 15 to 1567
Answer: position 2720 to 1557
Answer: position 1951 to 5824
Answer: position 3109 to 3112
Answer: position 1559 to 1170
Answer: position 1559 to 402
Answer: position 789 to 3500
Answer: position 3882 to 4655
Answer: position 4270 to 1176
Answer: position 1561 to 2340
Answer: position 398 to 3493
Answer: position 3105 to 3107
Answer: position 3109 to 5429
Answer: position 4657 to 15
Answer: position 3882 to 2724
Answer: position 5818 to 3495
Answer: position 1954 to 1565
Answer: position 1561 to 4276
Answer: position 1949 to 1952
Answer: position 2334 to 2721
Answer: position 3882 to 19
Answer: position 5433 to 1177
Answer: position 1946 to 1947
Answer: position 791 to 2718
Answer: position 4269 to 3118
Answer: position 5438 to 5439
Answer: position 5828 to 4657


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

...

Answer: position 408 to 3502
Answer: position 405 to 407
Answer: position 1563 to 3502
Answer: position 398 to 3113
Answer: position 3888 to 4662
Answer: position 405 to 2344
Answer: position 2341 to 5435
Answer: position 5434 to 1956
Answer: position 3882 to 5052
Answer: position 1568 to 405
Answer: position 397 to 400
Answer: position 2335 to 1178
Answer: position 792 to 1178
Answer: position 4655 to 5043
Answer: position 3492 to 18
Answer: position 1949 to 5435
Answer: position 5045 to 2341
Answer: position 3112 to 5440
Answer: position 3500 to 5442
Answer: position 18 to 3507
Answer: position 4657 to 12
Answer: position 2345 to 3499
Answer: position 1177 to 3505
Answer: position 5815 to 794
Answer: position 5823 to 5825
Answer: position 3891 to 22
Answer: position 2726 to 3114
Answer: position 1172 to 5824
Answer: position 3501 to 3115
Answer: position 3879 to 3882
Answer: position 5839 to 5841
Answer: position 2725 to 2727


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Answer: position 5826 to 3897
Answer: position 401 to 4278


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Answer: position 9 to 10
Answer: position 2724 to 4275
Answer: position 1177 to 5431
Answer: position 4269 to 1950
Answer: position 4278 to 4661
Answer: position 3500 to 3501
Answer: position 787 to 1946
Answer: position 4269 to 1563
Answer: position 396 to 3108
Answer: position 404 to 1956
Answer: position 2336 to 2337
Answer: position 20 to 22
Answer: position 5431 to 5433
Answer: position 3498 to 3114
Answer: position 5820 to 5821
Answer: position 788 to 1562
Answer: position 5827 to 406
Answer: position 15 to 1563
Answer: position 3883 to 5441
Answer: position 5816 to 5817
Answer: position 5436 to 3110
Answer: position 11 to 12
Answer: position 2718 to 1947
Answer: position 5817 to 3501
Answer: position 4656 to 3886
Answer: position 3882 to 794
Answer: position 401 to 2727
Answer: position 3110 to 790
Answer: position 3498 to 1949
Answer: position 3504 to 3506
Answer: position 1562 to 402
Answer: position 4265 to 3497
Answer: position 3501 to 3881
Answer: position 4661 to 5825
Answ